# Exploratory Data Analysis

The data used throughout this course is information on Formula 1 results, sourced from this Kaggle dataset: [Formula 1 World Championship (1950 - 2022)](https://www.kaggle.com/datasets/rohanrao/formula-1-world-championship-1950-2020).

The tables that have been downloaded are:

* Fact tables:
    * [lap_times.csv](./input/lap_times.csv): Provides lap times for each lap for each race by driver.
    * [pit_stops.csv](./input/pit_stops.csv): Contains information on pit stops for each race and driver.
    * [qualifying.csv](./input/qualifying.csv): Provides qualifying results for each race and driver.
    * [results.csv](./input/results.csv): Provides race results for every race and driver.
    
    
* Dim tables:
    * [circuits.csv](./input/circuits.csv): Lookup table providing further information for each `circuitId`.
    * [constructors.csv](./input/constructors.csv): Lookup table providing further information for each `constructorId`.
    * [drivers.csv](./input/drivers.csv): Lookup table providing further information for each `driverId`.
    * [races.csv](./input/races.csv): Lookup table providing further information for each `raceId`.
    * [status.csv](./input/status.csv): Lookup table providing descriptions for each status present in `results.csv`.
    
The aim of this script is to unite all fact tables with every related dim table to produce datasets for visualisation throughout this course.

In [1]:
import numpy as np
import pandas as pd   # Manipulation
pd.set_option('display.max_columns', None)
import datetime as dt # Conversion of mm:ss strings to seconds

# Read in datasets and drop/rename where appropriate
#   `time` variable is a string representing time in minutes:seconds
#   `duration` variable is a numeric representing time in seconds
#     Drop both in favour of `milliseconds` column
fct_lap_times    = pd.read_csv('./input/lap_times.csv').drop(columns = ['time'])
fct_pit_stops    = pd.read_csv('./input/pit_stops.csv').drop(columns = ['time', 'duration'])
fct_qualifying   = pd.read_csv('./input/qualifying.csv').drop(columns = ['qualifyId', 'number'])
fct_results      = pd.read_csv('./input/results.csv')
dim_circuits     = pd.read_csv('./input/circuits.csv').drop(columns = ['url', 'lat', 'lng'])
dim_constructors = pd.read_csv('./input/constructors.csv').drop(columns = ['url'])
dim_drivers      = pd.read_csv('./input/drivers.csv').drop(columns = ['dob', 'url'])
dim_races        = pd.read_csv('./input/races.csv').drop(columns = ['time', 'url'])
dim_status       = pd.read_csv('./input/status.csv')

## Processing lap_times.csv

In [2]:
# Glimpse available variables
fct_lap_times.head(3)

,raceId,driverId,lap,position,milliseconds
0,841,20,1,1,98109
1,841,20,2,1,93006
2,841,20,3,1,92713


In [3]:
# Join in on race and driver IDs
comb_lap_times = (
    fct_lap_times
        .merge(
            dim_races,
            how = 'left',
            on = 'raceId'
        )
        .merge(
            dim_drivers,
            how = 'left',
            on = 'driverId'
        )
        .drop(
            columns = ['raceId', 'driverId']
        )
)

comb_lap_times.head(3)

,lap,position,milliseconds,year,round,circuitId,name,date,driverRef,number,code,forename,surname,nationality
0,1,1,98109,2011,1,1,Australian Grand Prix,27/03/11,vettel,5,VET,Sebastian,Vettel,German
1,2,1,93006,2011,1,1,Australian Grand Prix,27/03/11,vettel,5,VET,Sebastian,Vettel,German
2,3,1,92713,2011,1,1,Australian Grand Prix,27/03/11,vettel,5,VET,Sebastian,Vettel,German


## Processing pit_stops.csv

In [4]:
# Glimpse available variables
fct_pit_stops.head(3)

,raceId,driverId,stop,lap,milliseconds
0,841,153,1,1,26898
1,841,30,1,1,25021
2,841,17,1,11,23426


In [5]:
# Join in on race and driver IDs
comb_pit_stops = (
    fct_pit_stops
        .merge(
            dim_races.rename(columns = {'name': 'gp_name'}),
            how = 'left',
            on = 'raceId'
        )
        .merge(
            dim_drivers,
            how = 'left',
            on = 'driverId'
        )
        .merge(
            dim_circuits.rename(columns = {'name': 'track_name'}),
            how = 'left',
            on = 'circuitId'
        )
        .drop(
            columns = ['raceId', 'driverId', 'circuitId']
        )
)

comb_pit_stops.head(3)

,stop,lap,milliseconds,year,round,gp_name,date,driverRef,number,code,forename,surname,nationality,circuitRef,track_name,location,country,alt
0,1,1,26898,2011,1,Australian Grand Prix,27/03/11,alguersuari,\N,ALG,Jaime,Alguersuari,Spanish,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,10
1,1,1,25021,2011,1,Australian Grand Prix,27/03/11,michael_schumacher,\N,MSC,Michael,Schumacher,German,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,10
2,1,11,23426,2011,1,Australian Grand Prix,27/03/11,webber,\N,WEB,Mark,Webber,Australian,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,10


## Processing qualifying.csv

In [6]:
# Glimpse available variables
fct_qualifying.head(3)

,raceId,driverId,constructorId,position,q1,q2,q3
0,18,1,1,1,1:26.572,1:25.187,1:26.714
1,18,9,2,2,1:26.103,1:25.315,1:26.869
2,18,5,1,3,1:25.664,1:25.452,1:27.079


In [7]:
# Join in on race, driver, and constructor IDs
comb_qualifying = (
    fct_qualifying
        .merge(
            dim_races.rename(columns = {'name': 'gp_name'}),
            how = 'left',
            on = 'raceId'
        )
        .merge(
            dim_drivers.rename(columns = {'nationality': 'driver_nationality'}),
            how = 'left',
            on = 'driverId'
        )
        .merge(
            dim_constructors,
            how = 'left',
            on = 'constructorId'
        )
        .merge(
            dim_circuits.rename(columns = {'name': 'track_name'}),
            how = 'left',
            on = 'circuitId'
        )
        .drop(
            columns = ['raceId', 'driverId', 'constructorId', 'circuitId']
        )
)

comb_qualifying.head(3)

,position,q1,q2,q3,year,round,gp_name,date,driverRef,number,code,forename,surname,driver_nationality,constructorRef,name,nationality,circuitRef,track_name,location,country,alt
0,1,1:26.572,1:25.187,1:26.714,2008,1,Australian Grand Prix,16/03/08,hamilton,44,HAM,Lewis,Hamilton,British,mclaren,McLaren,British,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,10
1,2,1:26.103,1:25.315,1:26.869,2008,1,Australian Grand Prix,16/03/08,kubica,88,KUB,Robert,Kubica,Polish,bmw_sauber,BMW Sauber,German,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,10
2,3,1:25.664,1:25.452,1:27.079,2008,1,Australian Grand Prix,16/03/08,kovalainen,\N,KOV,Heikki,Kovalainen,Finnish,mclaren,McLaren,British,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,10


In [8]:
# Qualifying times in this table are reported in strings with the format minutes:seconds.milliseconds.

# Define a function to convert these strings to seconds
def conv_to_seconds(string):
    timedeltas = pd.to_datetime(string, format="%M:%S.%f", errors = 'coerce') - pd.to_datetime('1/1/1900 00:00')
    return timedeltas / np.timedelta64(1, "s")

comb_qualifying[['q1', 'q2', 'q3']] = comb_qualifying[['q1', 'q2', 'q3']].apply(conv_to_seconds)

comb_qualifying.head(3)

,position,q1,q2,q3,year,round,gp_name,date,driverRef,number,code,forename,surname,driver_nationality,constructorRef,name,nationality,circuitRef,track_name,location,country,alt
0,1,86.572,85.187,86.714,2008,1,Australian Grand Prix,16/03/08,hamilton,44,HAM,Lewis,Hamilton,British,mclaren,McLaren,British,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,10
1,2,86.103,85.315,86.869,2008,1,Australian Grand Prix,16/03/08,kubica,88,KUB,Robert,Kubica,Polish,bmw_sauber,BMW Sauber,German,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,10
2,3,85.664,85.452,87.079,2008,1,Australian Grand Prix,16/03/08,kovalainen,\N,KOV,Heikki,Kovalainen,Finnish,mclaren,McLaren,British,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,10


## Processing results.csv

In [9]:
# Glimpse available variables
fct_results.head(3)

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1


In [10]:
# Join in on race, driver, and constructor IDs
comb_results = (
    fct_results
        .drop(columns = ['number'])
        .merge(
            dim_races.rename(columns = {'name': 'gp_name'}),
            how = 'left',
            on = 'raceId'
        )
        .merge(
            dim_drivers.rename(columns = {'nationality': 'driver_nationality'}),
            how = 'left',
            on = 'driverId'
        )
        .merge(
            dim_constructors,
            how = 'left',
            on = 'constructorId'
        )
        .merge(
            dim_circuits.rename(columns = {'name': 'track_name'}),
            how = 'left',
            on = 'circuitId'
        )
        .merge(
            dim_status,
            how = 'left',
            on = 'statusId'
        )
        .drop(
            columns = ['resultId', 'raceId', 'driverId', 'constructorId', 'circuitId', 'statusId', 'time']
        )
)

comb_results['fastestLapTime'] = comb_results['fastestLapTime'].apply(conv_to_seconds)
comb_results['fastestLapSpeed'] = comb_results['fastestLapSpeed'].replace({'\\N': np.nan}).astype(float)

comb_results.head(3)

,grid,position,positionText,positionOrder,points,laps,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,year,round,gp_name,date,driverRef,number,code,forename,surname,driver_nationality,constructorRef,name,nationality,circuitRef,track_name,location,country,alt,status
0,1,1,1,1,10.0,58,5690616,39,2,87.452,218.300,2008,1,Australian Grand Prix,16/03/08,hamilton,44,HAM,Lewis,Hamilton,British,mclaren,McLaren,British,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,10,Finished
1,5,2,2,2,8.0,58,5696094,41,3,87.739,217.586,2008,1,Australian Grand Prix,16/03/08,heidfeld,\N,HEI,Nick,Heidfeld,German,bmw_sauber,BMW Sauber,German,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,10,Finished
2,7,3,3,3,6.0,58,5698779,41,5,88.090,216.719,2008,1,Australian Grand Prix,16/03/08,rosberg,6,ROS,Nico,Rosberg,German,williams,Williams,British,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,10,Finished


## Save to CSVs

In [11]:
comb_lap_times.to_csv('./intermediate/lap_times_prepped.csv')
comb_pit_stops.to_csv('./intermediate/pit_stops_prepped.csv')
comb_qualifying.to_csv('./intermediate/qualifying_prepped.csv')
comb_results.to_csv('./intermediate/results_prepped.csv')